In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from math import sin, cos, sqrt, atan2, radians
import random
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool
import glob
import sys
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)
from utils.gsvload import GSVSummary
import concurrent.futures

# Goal:
1. Check downloaded data
2. Create the updated {city}_meta.csv file for each city 
3. the {city}_meta.csv should only include the downloade gsv and selected data in gsv_pano_label.csv
4. this meta file is used for segmentation and city never was project. Remaining downloaded data will stay in the gsv_path.csv file

In [63]:
META_FILE = "{citylower}_meta.csv"
city = "New York"
city_abbr = city.lower().replace(" ", "_")
meta_file= META_FILE.format(citylower = city_abbr.replace("_", ""))
citysummary = GSVSummary(city)
GSV_META_FOLDER = citysummary.metafolder

meta_df = pd.read_csv(os.path.join(citysummary.metafolder, meta_file), nrows= 1)

In [2]:
meta = {
    "path":"path to the gsv",
    "panoid":"unique identifier",
    "angle":"image angle",
    "size":"image size",
    "lat":"latitude",
    "lon":"longitude",
    "year":"year taken",
    "month":"month",
    "id":"image id",
    "dist_hav":"distance from the city center", 
    "h3_res8":"h3 level 8",
    "h3_res9":"h3 level 9"
}

# 1. Check all data availability

In [3]:

serviceaccount = "../../google_drive_personal.json"
import gspread

# from oauth2client.service_account import ServiceAccountCredentials
gc = gspread.service_account(filename=serviceaccount)


def read_url(url, SHEET_NAME):
    SHEET_ID = url.split("/")[5]
    spreadsheet = gc.open_by_key(SHEET_ID)
    worksheet = spreadsheet.worksheet(SHEET_NAME)
    rows = worksheet.get_all_records()
    df_spread = pd.DataFrame(rows)
    return df_spread, worksheet


url = "https://docs.google.com/spreadsheets/d/1o5gFmZPUoDwrrbfE6M26uJF3HnEZll02ivnOxP6K6Xw/edit?usp=sharing"
SHEETNAME = "select_city"
city_meta, other_worksheet = read_url(url, SHEETNAME)
city_meta = city_meta[city_meta['City']!=''].reset_index(drop = True)

In [4]:
ROOT = "/lustre1/g/geog_pyloo"


In [4]:
def get_path(subfolder):
    files = glob.glob(os.path.join(subfolder, "*/*/*.jpg"))
    return files


def get_file_size(file):
    try:
        return file, os.path.getsize(file)
    except OSError as e:
        print(f"Error: {e}")
        return file, None


class GSVnew(GSVSummary):
    def __init__(self, city):
        super().__init__(city)

    def get_gsv_file_size(self, gsv_meta_df):
        gsvpath = gsv_meta_df.copy()
        files = gsvpath["path"].values
        # Store file sizes in a dictionary
        file_sizes = {}
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            # The map method helps maintain the order of results
            results = executor.map(get_file_size, files)

            for file, size in results:
                if size is not None:
                    file_sizes[file] = size
                else:
                    file_sizes[file] = 0  # the file is removed

        gsvpath["size"] = gsvpath["path"].apply(lambda x: file_sizes[x])
        # gsvpath.to_csv(os.path.join(self.metafolder, "gsv_path.csv"), index=False)
        return gsvpath

## 2.Check all downloaded GSV
1. Count number of GSV downloaded
2. Generate the gsv_path.csv file for all downloaded gsv

In [5]:
import datetime

today = datetime.datetime.today().strftime("%Y-%m-%d")
gsv_path_file = "gsv_path.csv"

In [17]:
# current being transfered, can summarize later
city_to_transfer = ['jakarta', 
                    'lima', # fully transfered
                    'metromanila', # fully transfered
                    'nagoya',# fully transfered
                    'riodejaneiro',# fully transfered
                    'milan', # 
                    'bangkok',
                    'toronto',
                    'kualalumpur'
                   ]
new_cities = city_meta[city_meta["GSV Downloaded"]==""]["City"].unique()

In [7]:
new_cities = ['Milan', 'Bangkok'] # temporary, given the previous download was suspended in the middle

In [8]:
# This function checks whether there are more gsv downloaded but not included in the gsv_path file
noimage = []
for city in tqdm(new_cities):
    print(city)
    citysummary = GSVnew(city)
    gsvpath = citysummary.load_finshed_gsv()
    gsvpath.to_csv(os.path.join(citysummary.metafolder, "gsv_path.csv"), index = False)
    # if gsvpath.shape[0]>0:
#         # load previous gsv path
#         try:
#             gsvpath_original = pd.read_csv(os.path.join(citysummary.metafolder, "gsv_path.csv"))
#             print(gsvpath_original.shape[0], ": original")
#             remain = gsvpath[
#                 gsvpath["panoid"].isin(gsvpath_original["panoid"]) == False
#             ].reset_index(drop=True)
#             if remain.shape[0] > 0:
#                 gsvpath_update = citysummary.get_gsv_file_size(remain)

#                 gsvpath_new = pd.concat([gsvpath_original, gsvpath_update], axis=0)
#                 print(gsvpath_new.shape[0], ": original+new")
#                 gsvpath_new.to_csv(
#                     os.path.join(citysummary.metafolder, "gsv_path.csv"), index=False
#                 )
#             else:
#                 print("no new gsv")
#             # os.remove(os.path.join(citysummary.metafolder, gsv_path_file))
#         except:
#             print("no path file exists")
#         gsvpath_update = citysummary.get_gsv_file_size(remain) # then it will include the file size
#         gsvpath_update.to_csv(
#                 os.path.join(citysummary.metafolder, "gsv_path.csv"), index=False
#             )
#     else:
#         print(city, " has no images downloaded")
#         noimage.append(city)

  0%|          | 0/2 [00:00<?, ?it/s]

Milan



 50%|█████     | 1/2 [00:09<00:09,  9.03s/it]

Bangkok



100%|██████████| 2/2 [00:33<00:00, 16.92s/it]


In [8]:
road_result = []
pano_result = []
gsv_result = []

for city in tqdm(new_cities):
    result = GSVnew(city)
    try:
        road_result.append(result.check_road())
    except:
        road_result.append(0)
    try:
        pano_result.append(result.check_pano()[1])
    except:
        pano_result.append(0)
    try:
        gsvdf = result.load_gsv_path()
        n = gsvdf.shape[0]
        gsv_result.append(n)
    except:
        gsv_result.append(0)

100%|██████████| 1/1 [00:02<00:00,  2.48s/it]


In [9]:
inspectdf = pd.DataFrame(
    {
        "city": new_cities,
        "num_panoid": pano_result,
        "road": road_result,
        "GSV Downloaded": gsv_result,
    }
)
inspectdf

,city,num_panoid,road,GSV Downloaded
0,Vienna,1166126,1,319435


In [10]:

city_meta = city_meta.set_index('City')
for city in inspectdf.city.unique():
    temp = inspectdf[inspectdf['city']==city].reset_index(drop = True)
    city_meta.at[city, 'num_panoid'] = temp['num_panoid'].values[0]
    city_meta.at[city,  'road'] = temp['road'].values[0]
    city_meta.at[city,  'GSV Downloaded'] = temp['GSV Downloaded'].values[0]

In [11]:
city_meta = city_meta.reset_index()
city_meta = city_meta.astype(str)
city_meta = city_meta[city_meta['City']!='nan'].reset_index(drop = True)

In [12]:
# write to google sheet
other_worksheet.update(
    [city_meta.columns.values.tolist()] + city_meta.values.tolist()
)

{'spreadsheetId': '1o5gFmZPUoDwrrbfE6M26uJF3HnEZll02ivnOxP6K6Xw',
 'updatedRange': 'select_city!A1:M160',
 'updatedRows': 160,
 'updatedColumns': 13,
 'updatedCells': 2080}

# Create final pano meta file for analysis


In [13]:
# cityabbr = city.lower().replace(" ", "")
META_FILE = '{cityabbr}_meta.csv'
META_FOLDER = "gsvmeta"
GSVROOT = "/lustre1/g/geog_pyloo/05_timemachine/GSV/gsv_rgb/"
city_meta['GSV Downloaded'] = city_meta['GSV Downloaded'].astype(int)
cityls = city_meta[city_meta['GSV Downloaded']>0]['City'].unique()


In [14]:
for city in tqdm(['Vienna']): 
    # Step 0: check whether the meta data already exists
    cityabbr = city.lower().replace(" ", "")
    metafiles = os.listdir(os.path.join(GSVROOT, cityabbr, META_FOLDER))
    meta_file = META_FILE.format(cityabbr = cityabbr)
    
        # Step 1: check remaining data to add to the path
    citysummary = GSVSummary(city)
    gsvpath = pd.read_csv(os.path.join(citysummary.metafolder, "gsv_path.csv"))
    print(gsvpath.head(1))
    path_sample = gsvpath['path'].values[0]
    if "/data/" in path_sample:
        print(path_sample)
        print("need to correct the path")
        gsvpath['path'] = gsvpath['path'].apply(lambda x: x.replace("./data/", "/lustre1/g/geog_pyloo/05_timemachine/GSV/"))
        gsvpath.to_csv(os.path.join(citysummary.metafolder, "gsv_path.csv"), index = False)

    else:
        print("Path is already correct")
        print(gsvpath.shape[0], ": original")
    
    if meta_file in metafiles:
        print(f"CITY {city} meta file already exists")

    else:
        print("Meta file missing, now processing: ", city)
        
        # Step 2: merge the meta file and save file
        metadf_update = citysummary.merge_meta(sel = True)
        metadf_update.drop("geometry", axis = 1).to_csv(os.path.join(GSVROOT, cityabbr, META_FOLDER, meta_file), index = False)
        print("final saved file number of rows: ", metadf_update.shape[0])
        import gc
        gc.collect()


  0%|          | 0/1 [00:00<?, ?it/s]

                                                path                  panoid
0  /lustre1/g/geog_pyloo/05_timemachine/GSV/gsv_r...  gQwGknUmHhpaulRLzw_1AQ
Path is already correct
319435 : original
Meta file missing, now processing:  Vienna


100%|██████████| 1/1 [00:06<00:00,  6.79s/it]

final saved file number of rows:  347836


In [15]:
metadf_update.drop("geometry", axis = 1).drop_duplicates("path").to_csv(os.path.join(GSVROOT, cityabbr, META_FOLDER, meta_file), index = False)

In [87]:
# clean all the meta file to drop geometry and drop duplicates
for city in tqdm(cityls):
    cityabbr = city.lower().replace(" ", "")
    metafiles = os.listdir(os.path.join(GSVROOT, cityabbr, META_FOLDER))
    meta_file = META_FILE.format(cityabbr = cityabbr)
    
        # Step 1: check remaining data to add to the path
    citysummary = GSVSummary(city)
    metadf = pd.read_csv(os.path.join(GSVROOT, cityabbr, META_FOLDER, meta_file))
    if 'geometry' in metadf.columns:
        metadf.drop('geometry', axis = 1).drop_duplicates("path").to_csv(os.path.join(GSVROOT, cityabbr, META_FOLDER, meta_file), index = False)
    else:
        print("data fixed, continue")
        continue

  1%|          | 1/126 [00:02<04:37,  2.22s/it]

data fixed, continue


  2%|▏         | 2/126 [00:04<04:57,  2.40s/it]

data fixed, continue


  2%|▏         | 3/126 [00:05<03:09,  1.54s/it]

...

data fixed, continue


 99%|█████████▉| 125/126 [04:38<00:01,  1.52s/it]

data fixed, continue


100%|██████████| 126/126 [04:41<00:00,  2.23s/it]

data fixed, continue
